# COCO dataset -- Dowload and preparation

## 1. Download the dataset
### 1.1 Download and extract the annotation files
### 1.2 Selected the images of interest and the corresponding labels

## 2. Image preparation
...

## 1. Download the dataset

In [73]:
from imutils import paths
import numpy as np
import os

### 1.1 Download and extract the annotation files

In [2]:
url = "http://images.cocodataset.org/annotations/annotations_trainval2014.zip"

In [4]:
!wget $url

--2022-09-16 11:35:34--  http://images.cocodataset.org/annotations/annotations_trainval2014.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.41.1
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.41.1|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252872794 (241M) [application/zip]
Saving to: ‘annotations_trainval2014.zip’

inval2014.zip         1%[                    ]   4,32M   757KB/s    eta 5m 36s ^C


In [10]:
annDir = "./dataset/coco/"
annFileURLs = ["http://images.cocodataset.org/annotations/annotations_trainval2014.zip",
                "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"]

if not os.path.isdir(annDir):
    os.makedirs(annDir)
    
def download_annotations(URL, force_download=False):
    
    download_ann = True
    fileName = URL.split(os.sep)[-1]
    dataType = fileName.split("_")[-1].split(".")[0][-4:]  
    zipPath = os.sep.join([".", fileName])
    dstPath = os.sep.join([annDir, "annotations"])
    
    print(f"[INFO]  Attempting to download {fileName}...")
    
    if os.path.isdir(dstPath):
        files = list(paths.list_files(annDir, contains=dataType))
        if files:
            download_ann = False
            print(f"[INFO] Annotations {dataType} already downloaded in {annDir}.")
    else: 
        os.makedirs(dstPath)
    
    if force_download:
        download_ann = True
        print("[INFO] Forcing download...")
        
    if download_ann:
        if not os.path.exists(zipPath):
            !wget $URL
        !unzip $zipPath 
        !mv  ./annotations/* $dstPath
        !rm -r ./annotations
        !rm $zipPath
    
    print(f"[INFO] Annotations downloaded at {dstPath}.")
        

for url in annFileURLs:
    download_annotations(url)

[INFO]  Attempting to download annotations_trainval2014.zip...
Archive:  ./annotations_trainval2014.zip
  inflating: annotations/instances_train2014.json  
  inflating: annotations/instances_val2014.json  
  inflating: annotations/person_keypoints_train2014.json  
  inflating: annotations/person_keypoints_val2014.json  
  inflating: annotations/captions_train2014.json  
  inflating: annotations/captions_val2014.json  
[INFO] Annotations downloaded at ./dataset/coco//annotations.
[INFO]  Attempting to download annotations_trainval2017.zip...
Archive:  ./annotations_trainval2017.zip
  inflating: annotations/instances_train2017.json  
  inflating: annotations/instances_val2017.json  
  inflating: annotations/captions_train2017.json  
  inflating: annotations/captions_val2017.json  
  inflating: annotations/person_keypoints_train2017.json  
  inflating: annotations/person_keypoints_val2017.json  
[INFO] Annotations downloaded at ./dataset/coco//annotations.


### 1.2 Selected the images of interest and the corresponding labels

We select only images containing persons. On those pics we apply the following rules: 
- Select  all keyPoints (visible and not visible) 
- Select the most centered person on the basis of the visible keyPoints 
- Check that the selected person is "big" enough inside the image size 
- Save the image and the corresponding keypoint 

In [17]:
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
import skimage.io as io

In [144]:
dataDir = "./dataset/coco/"

def select_imgs_with_persons(datatype):
    """ 
        Selects all the images containing persons from the coco datasets.
        
        Params
        ------
        datatype: str
            Data source. Options: 'train2014', 'val2014', 'train2017', 'val2017'.
            
        Returns
        -------
        imgs: list(dict)
            List of dict containing the images information.
        coco_kps: pycocotools.coco.COCO
            Object containing all the keypoints information of the selected images.
    """
    annFile = f"{dataDir}/annotations/instances_{dataType}.json"
    coco_anns = COCO(annFile)
    catIds = coco_anns.getCatIds(catNms=['person'])
    imgIds = coco_anns.getImgIds(catIds=catIds)
    print(f"[INFO] Found {len(imgIds)} raw images...")
    
    kptsFile = f"{dataDir}/annotations/person_keypoints_{dataType}.json"
    coco_kps = COCO(kptsFile)
    imgs = coco_anns.loadImgs(imgIds)
    
    return imgs, coco_kps


def get_centre_of_mass(keypoints_list):
    """
        Returns the centers of mass of a given list of keypoints.
        
        Params
        ------
        keypoints_list: np.array
            Array of shape (# of persons, # of joints, 3) containing all the keypoints coordinates.
        
        Returns
        -------
        centers: np.array
            Array of shape (# of persons, 2) with the coordinates of the persons centre of mass.
    """
    centers = []
    
    for skeleton in keypoints_list:
        vis_joints = skeleton[:,-1] == 2
        centers.append(skeleton[vis_joints].mean(axis=0)[:2])
    
    centers = np.array(centers)
    
    return centers


def get_most_centered_keypoints(coco_kps, imgId, img_size):
    """
        Returns the keypoints coordinates of the most centered person.
        
        Params
        ------
        coco_kps: pycocotools.coco.COCO
            Object containing all the keypoints information of the selected images.
        imgId: str
            Image Id.
        img_size: tuple
            Image height and width in pixels.
        
        Returns
        -------
        most_centered: np.array
            Array of shape (# of joints, 3)with the coordinates of the most centered person.
    """
    annIds = coco_kps.getAnnIds(imgIds=imgId, catIds=[1], iscrowd=False)
    anns = coco_kps.loadAnns(annIds)
    keypoints_list = []
    
    for ann in anns:

        keypoints = np.array(ann["keypoints"], dtype=np.float32).reshape(-1, 3)
        keypoints[:,:2] = keypoints[:,:2] / img_size[::-1]
        
        if keypoints[:,:2].sum() == 0:
            continue
        
        keypoints_list.append(keypoints)
        
    keypoints_list = np.array(keypoints_list)
    centers = get_centre_of_mass(keypoints_list)
    
    # get most centered skeleton
    most_centered = np.linalg.norm(centers - (.5, .5), axis=1).argmin() 
    most_centered = keypoints_list[most_centered]
    
    return most_centered
    

In [145]:
imgs, kps = select_imgs_with_persons("val2017")

i = 1

imgId, img_size = imgs[i]["id"], (imgs[i]["height"], imgs[i]["width"])
c_keypts = get_most_centered_keypoints(kps, imgId, img_size)
c_keypts

loading annotations into memory...
Done (t=0.47s)
creating index...
index created!
[INFO] Found 2693 raw images...
loading annotations into memory...
Done (t=0.59s)
creating index...
index created!


array([[0.        , 0.        , 0.        ],
       [0.61875   , 0.26041666, 2.        ],
       [0.        , 0.        , 0.        ],
       [0.678125  , 0.36666667, 2.        ],
       [0.8328125 , 0.35833332, 2.        ],
       [0.63125   , 0.76666665, 2.        ],
       [0.790625  , 0.55833334, 2.        ],
       [0.328125  , 0.7416667 , 2.        ],
       [0.65625   , 0.5708333 , 2.        ],
       [0.2390625 , 0.40208334, 2.        ],
       [0.5359375 , 0.54375   , 2.        ],
       [0.        , 0.        , 0.        ],
       [0.6828125 , 0.83958334, 2.        ],
       [0.3078125 , 0.86041665, 2.        ],
       [0.4125    , 0.63125   , 1.        ],
       [0.        , 0.        , 0.        ],
       [0.5390625 , 0.9895833 , 1.        ]], dtype=float32)

In [ ]:
# TODO
# Check that the function get_most_centered_keypoints is wortking properly
